## Notebook Description

This notebook **merges and preprocesses Great Tit metadata files into a single cleaned dataset for deep learning**.

**Main tasks:**

1. **Loads three metadata files**: `great-tit-hits.csv`, `main.csv`, and `feature_vectors.csv` from the original metadata directory

2. **Merges tables**: Combines `great-tit-hits` with `main` using ID/pnum fields, then integrates 384-dimensional audio embeddings

3. **Filters columns**: Retains only essential fields - filename, song type (`class_id`), bird ID, and individual label (`father`)

4. **Removes unlabeled data**: Filters out records where `father` field is NaN, keeping only individually-labeled birds

5. **Dataset statistics**: Shows 242 unique individuals and 1,307 unique song types from 74,048 total audio samples

6. **Saves final dataset**: Exports cleaned metadata as `final_greatTit_metadata.csv` containing:
   - Audio filename (without .wav extension)
   - Song classification ID
   - Bird ID and father label
   - 384-dimensional embedding features

**Result**: Clean, merged dataset ready for individual bird classification with both metadata and pre-computed audio embeddings.

This notebook loads the 3 main metadata files of the great tit and merges them into a single file. The columns are filtered to retain only the necessary information for a deep learning classifier.

Additionally, only the data labeled by individual (father) is selected.

The output will be a file containing the associated audio file name (without the .wav), the type of song, the father label, and a 384-dimensional embedding for each song.

# Load the data into Python

In [4]:
from pathlib import Path
import pandas as pd

# 1. Path.cwd() is the current working directory
cwd = Path.cwd()

# 2. Move two levels up to reach the root of the project
project_root = cwd.parents[1]  

# 3. Build the path to the CSV files clearly
csv_path = project_root / 'Original_metadata' / 'GreatTit_metadata' / 'great-tit-hits.csv'
main_path = project_root / 'Original_metadata' / 'GreatTit_metadata' / 'main.csv'
embeddings_path = project_root / 'Original_metadata' / 'GreatTit_metadata' / 'feature_vectors.csv'


# 4. Read the CSV
great_tit_hits = pd.read_csv(csv_path)
main = pd.read_csv(main_path)
embeddings = pd.read_csv(embeddings_path)


print(csv_path) 
print (main_path)
print (embeddings_path)

/teamspace/studios/this_studio/Original_metadata/GreatTit_metadata/great-tit-hits.csv
/teamspace/studios/this_studio/Original_metadata/GreatTit_metadata/main.csv
/teamspace/studios/this_studio/Original_metadata/GreatTit_metadata/feature_vectors.csv


# Merge the tables

Use the pandas merge function to join both tables based on the columns ID and pnum. In this case, a left join is used to keep all records from great_tit_hits.

In [6]:
# Merge the tables using ID and pnum
merged_data = pd.merge(great_tit_hits, main, left_on='ID', right_on='pnum', how='left')

# Check the first rows of the merged DataFrame
merged_data


,Unnamed: 0,ID,class_id,datetime,start,end,length_s,bit_rate,sample_rate,lower_freq,...,habitat_type,soil_type,recorded,total_recordings,missing_recordings,first_recorded,last_recorded,delay,n_vocalisations,repertoire_size
0,20201B161_20200418_040000_130451999,20201B161,20201B161_1,2020-04-18 04:45:17.749979167,130451999,130569739,2.452925,768000,22050,3586,...,5.0,2.0,True,21.0,0.0,2020-04-18,2020-04-20,3.0,33.0,3.0
1,20201B161_20200418_050000_11502511,20201B161,20201B161_1,2020-04-18 05:03:59.635645833,11502511,11636554,2.792608,768000,22050,3522,...,5.0,2.0,True,21.0,0.0,2020-04-18,2020-04-20,3.0,33.0,3.0
2,20201B161_20200418_050000_13585746,20201B161,20201B161_1,2020-04-18 05:04:43.036375000,13585746,13721600,2.830295,768000,22050,3673,...,5.0,2.0,True,21.0,0.0,2020-04-18,2020-04-20,3.0,33.0,3.0
3,20201B161_20200418_050000_13982440,20201B161,20201B161_1,2020-04-18 05:04:51.300833333,13982440,14121918,2.905805,768000,22050,3586,...,5.0,2.0,True,21.0,0.0,2020-04-18,2020-04-20,3.0,33.0,3.0
4,20201B161_20200418_050000_20011659,20201B161,20201B161_1,2020-04-18 05:06:56.909562500,20011659,20212723,4.188844,768000,22050,3630,...,5.0,2.0,True,21.0,0.0,2020-04-18,2020-04-20,3.0,33.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109958,20221W99_20220416_060000_7793664,20221W99,20221W99_0,2022-04-16 06:02:42.368000000,7793664,7872512,1.642676,768000,22050,2410,...,5.0,3.0,True,30.0,4.0,2022-04-10,2022-04-16,-5.0,303.0,7.0
109959,20221W99_20220416_060000_8046592,20221W99,20221W99_0,2022-04-16 06:02:47.637333333,8046592,8148992,2.133333,768000,22050,2764,...,5.0,3.0,True,30.0,4.0,2022-04-10,2022-04-16,-5.0,303.0,7.0
109960,20221W99_20220416_060000_8297472,20221W99,20221W99_0,2022-04-16 06:02:52.864000000,8297472,8367104,1.450703,768000,22050,2732,...,5.0,3.0,True,30.0,4.0,2022-04-10,2022-04-16,-5.0,303.0,7.0
109961,20221W99_20220416_060000_8514048,20221W99,20221W99_0,2022-04-16 06:02:57.376000000,8514048,8585216,1.482676,768000,22050,2652,...,5.0,3.0,True,30.0,4.0,2022-04-10,2022-04-16,-5.0,303.0,7.0


# Filter the necessary columns

Once merged, only the columns relevant for the analysis are selected, such as class_id (type of song) and father (individual).

In [8]:
# Filter the necessary columns, including the first column
filtered_data = merged_data[['Unnamed: 0', 'class_id', 'ID','father' ]]

filtered_data


,Unnamed: 0,class_id,ID,father
0,20201B161_20200418_040000_130451999,20201B161_1,20201B161,te80930
1,20201B161_20200418_050000_11502511,20201B161_1,20201B161,te80930
2,20201B161_20200418_050000_13585746,20201B161_1,20201B161,te80930
3,20201B161_20200418_050000_13982440,20201B161_1,20201B161,te80930
4,20201B161_20200418_050000_20011659,20201B161_1,20201B161,te80930
...,...,...,...,...
109958,20221W99_20220416_060000_7793664,20221W99_0,20221W99,NaN
109959,20221W99_20220416_060000_8046592,20221W99_0,20221W99,NaN
109960,20221W99_20220416_060000_8297472,20221W99_0,20221W99,NaN
109961,20221W99_20220416_060000_8514048,20221W99_0,20221W99,NaN


# Integrate the embeddings

In [9]:
final_data = pd.concat([filtered_data, embeddings], axis=1)
final_data

,Unnamed: 0,class_id,ID,father,0,1,2,3,4,5,...,374,375,376,377,378,379,380,381,382,383
0,20201B161_20200418_040000_130451999,20201B161_1,20201B161,te80930,0.009118,-0.039894,0.019366,0.012394,0.025821,0.041465,...,-0.087559,0.003942,-0.039138,0.045033,-0.026501,0.069331,0.023100,-0.007022,-0.077677,0.004901
1,20201B161_20200418_050000_11502511,20201B161_1,20201B161,te80930,0.015071,-0.094309,0.012136,-0.006451,-0.007025,0.044730,...,-0.089846,0.006436,-0.055178,0.035964,-0.009658,0.037771,0.058808,-0.075454,-0.059461,0.011810
2,20201B161_20200418_050000_13585746,20201B161_1,20201B161,te80930,0.012520,-0.037601,0.030848,0.005622,0.046687,0.054802,...,-0.102847,-0.004241,-0.029241,0.051174,-0.037412,0.071012,0.019209,0.017237,-0.096958,-0.021968
3,20201B161_20200418_050000_13982440,20201B161_1,20201B161,te80930,0.017553,-0.055216,0.018768,-0.007323,0.017101,0.049850,...,-0.083391,0.001441,-0.029482,0.050818,-0.029794,0.038637,0.022456,-0.026888,-0.061264,-0.019537
4,20201B161_20200418_050000_20011659,20201B161_1,20201B161,te80930,0.033305,-0.048789,0.009809,0.000716,0.016080,0.029543,...,-0.081079,0.004624,-0.048797,0.047911,-0.036521,0.064695,0.030184,-0.053446,-0.084810,-0.037440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109958,20221W99_20220416_060000_7793664,20221W99_0,20221W99,NaN,0.048830,0.057714,0.030472,0.058629,0.012296,0.136139,...,-0.041955,0.005642,-0.031123,0.024130,0.054855,0.069271,0.086320,0.095439,-0.038689,0.003438
109959,20221W99_20220416_060000_8046592,20221W99_0,20221W99,NaN,0.011747,0.021026,0.018420,0.060821,0.009977,0.138268,...,-0.061360,-0.002816,-0.009603,0.034710,0.064480,0.089752,0.067404,0.073442,-0.037955,-0.038878
109960,20221W99_20220416_060000_8297472,20221W99_0,20221W99,NaN,0.025707,0.004182,0.016254,0.056812,0.012120,0.117936,...,-0.044460,-0.000674,-0.050931,0.039708,0.071608,0.080539,0.046728,0.081509,-0.029340,-0.029094
109961,20221W99_20220416_060000_8514048,20221W99_0,20221W99,NaN,0.059377,0.022741,0.012474,0.061463,0.012822,0.132026,...,-0.027559,0.013322,-0.045885,0.043523,0.089316,0.057430,0.057906,0.046405,-0.032163,0.001651


# Remove Nan from the column father. (Not all individuals are labeled)

In [10]:
# Remove rows where 'father' is NaN
final_data = final_data.dropna(subset=['father'])

# Total number of unique individuals
total_individuals = final_data['father'].nunique()
print(f"Total number of unique individuals: {total_individuals}")

# Total number of unique song types
total_types = final_data['class_id'].nunique()
print(f"Total number of unique song types: {total_types}")

final_data

Total number of unique individuals: 242
Total number of unique song types: 1307


,Unnamed: 0,class_id,ID,father,0,1,2,3,4,5,...,374,375,376,377,378,379,380,381,382,383
0,20201B161_20200418_040000_130451999,20201B161_1,20201B161,te80930,0.009118,-0.039894,0.019366,0.012394,0.025821,0.041465,...,-0.087559,0.003942,-0.039138,0.045033,-0.026501,0.069331,0.023100,-0.007022,-0.077677,0.004901
1,20201B161_20200418_050000_11502511,20201B161_1,20201B161,te80930,0.015071,-0.094309,0.012136,-0.006451,-0.007025,0.044730,...,-0.089846,0.006436,-0.055178,0.035964,-0.009658,0.037771,0.058808,-0.075454,-0.059461,0.011810
2,20201B161_20200418_050000_13585746,20201B161_1,20201B161,te80930,0.012520,-0.037601,0.030848,0.005622,0.046687,0.054802,...,-0.102847,-0.004241,-0.029241,0.051174,-0.037412,0.071012,0.019209,0.017237,-0.096958,-0.021968
3,20201B161_20200418_050000_13982440,20201B161_1,20201B161,te80930,0.017553,-0.055216,0.018768,-0.007323,0.017101,0.049850,...,-0.083391,0.001441,-0.029482,0.050818,-0.029794,0.038637,0.022456,-0.026888,-0.061264,-0.019537
4,20201B161_20200418_050000_20011659,20201B161_1,20201B161,te80930,0.033305,-0.048789,0.009809,0.000716,0.016080,0.029543,...,-0.081079,0.004624,-0.048797,0.047911,-0.036521,0.064695,0.030184,-0.053446,-0.084810,-0.037440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109380,20221W97_20220422_040000_152319488,20221W97_2,20221W97,ty68763,0.098876,-0.047654,-0.041847,-0.091682,0.030961,-0.063485,...,-0.018623,0.016563,-0.020582,0.012203,0.067731,-0.005839,-0.104200,-0.018688,-0.058134,0.017441
109381,20221W97_20220422_040000_155583360,20221W97_2,20221W97,ty68763,0.084930,-0.041180,-0.024030,-0.067535,0.050401,-0.039809,...,-0.021963,0.022392,0.002381,0.009173,0.070520,0.001552,-0.102922,0.016977,-0.055198,0.025448
109382,20221W97_20220422_040000_156154752,20221W97_2,20221W97,ty68763,0.093182,-0.046198,-0.039534,-0.060609,-0.014612,-0.055344,...,-0.034893,0.018837,-0.019449,0.005713,0.081382,-0.011172,-0.085339,-0.010182,-0.070577,0.012723
109383,20221W97_20220422_040000_156814848,20221W97_2,20221W97,ty68763,0.081777,-0.047179,-0.021467,-0.068880,0.017466,-0.053362,...,-0.052280,0.024417,-0.005323,-0.004866,0.073054,-0.003349,-0.106902,-0.014925,-0.055606,0.021484


In [11]:
# Save the final_data DataFrame to a CSV file

output_metadata_path = project_root / 'Output_metadata' / 'GreatTit_metadata' / 'final_greatTit_metadata.csv'
final_data.to_csv(output_metadata_path, index=False)

print("File successfully saved as final_greatTit_metadata.csv")


File successfully saved as final_greatTit_metadata.csv
